In [1]:
import os
import json
import re
import string
import pandas as pd
import numpy as np
from libvoikko import Voikko
from gensim.models import FastText
from gensim.models import Word2Vec

In [2]:
paths = ["data/kho/" + x for x in os.listdir("data/kho")]
paths += ["data/kko/" + x for x in os.listdir("data/kko")]
paths += ["data/sd/" + x for x in os.listdir("data/sd")]

In [3]:
data = []

for i in range(0, len(paths)):
    with open(paths[i], 'r') as f:
        data += json.load(f)
        
df = pd.DataFrame(data)
del data

In [4]:
df.sort_values("year", inplace = True)
df.reset_index(inplace = True)
df.drop("index", 1, inplace = True)

In [5]:
def clean(s):
    s = re.sub("\n", " ", s)
    s = re.sub("  ", " ", s)
    s = s.strip()
    return(s)

def remove_punct(s):
    s = s.translate(str.maketrans('', '', string.punctuation + string.digits))
    s = re.sub("§(\S+)?", "", s)
    s = re.sub(" +", " ", s)
    s = re.sub("(^| ).( |$)", " ", s)
    s = re.sub("–", "", s)
    s = s.strip()
    s = s.lower()
    return(s)

def tokenize_sentences(text, voikko):
    return [s.sentenceText.strip() for s in voikko.sentences(text)]

In [6]:
voikko = Voikko("fi")

In [13]:
texts = list(df['text'])

sentences = []
for i in range(0, len(texts)):
    tokens = tokenize_sentences( clean(texts[i]), voikko )
    sentences += [remove_punct(t) for t in tokens]

del texts

In [50]:
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_fasttext.html#sphx-glr-auto-examples-tutorials-run-fasttext-py
model_ft = FastText(texts, size = 200, window = 5, min_count = 5)

In [51]:
print(model_ft)

FastText(vocab=210918, size=200, alpha=0.025)


In [ ]:
model_ft.save("models/fasttext_doc")

In [12]:
model_w2v = Word2Vec(texts, size = 200, window = 5, min_count = 5)

In [13]:
print(model_w2v)

Word2Vec(vocab=210918, size=200, alpha=0.025)


In [15]:
model_w2v.save("models/w2v_doc")